# EDA Tablas Meta
---
tras nuestro analisis a las distintas tablas de datos, nos dimos cuenta de la similaridad en estructura de cada una de las tablas, por lo que crearemos un EDA modelo donde respetaremos los pasos para automatizar este proceso.

### Importacion de librerias

In [322]:
import pandas as pd #utilizaremos pandas para el mejor manejo de tablas
import numpy as np #utilizaremos numpy para un mejor tratamiento de los datos
import matplotlib.pyplot as plt
import seaborn as sns # matplotlib y seaborn seran usados para graficar
from pyspark.sql import SparkSession #pyspark lo usaremos como opcion al gran requerimiento de procesamiento de datos

# Crea una sesión de Spark
spark = SparkSession.builder \
    .appName("MiAplicacionSpark") \
    .getOrCreate()
#importamos las funciones sql de pyspark para el manejo de columnas y valores
from pyspark.sql.functions import col, sum, when, size, length, to_date, from_unixtime, year, regexp_replace, avg, lit

## Paso 1: Extraemos archivo.
---

In [323]:
df = spark.read.json('../datasets/meta_Digital_Music.json', encoding='UTF-8')
#observamos la cantidad de filas
df.count()

74347

In [324]:
df.show()

+--------------------+--------------------+----------+--------------------+--------+----+--------------------+-------+-------+---+--------------------+--------------------+--------------------+---------+--------------------+------------+-----+-----+--------------------+
|            also_buy|           also_view|      asin|               brand|category|date|         description|details|feature|fit|            imageURL|     imageURLHighRes|            main_cat|    price|                rank|similar_item|tech1|tech2|               title|
+--------------------+--------------------+----------+--------------------+--------+----+--------------------+-------+-------+---+--------------------+--------------------+--------------------+---------+--------------------+------------+-----+-----+--------------------+
|[B000002UEN, B000...|[B000002UEN, B000...|0001377647| John Michael Talbot|      []|    |                  []|   null|     []|   |                  []|                  []|<img src="https

## Paso 2: Manejo de Duplicados.
---

In [325]:
num_filas_duplicadas = df.count() - df.dropDuplicates().count()

# Imprime el número de filas duplicadas
print("Número de filas duplicadas:", num_filas_duplicadas)
print(f"Porcentaje de filas duplicadas: {num_filas_duplicadas / df.count()}")
df = df.dropDuplicates()

Número de filas duplicadas: 8334
Porcentaje de filas duplicadas: 0.11209598235302029


Eliminamos los duplicados ya que se repiten los valores en todas las columnas el mismo valor, con valores como also_buy que implican que es exactamente lo mismo

## Paso 3: Manejo de Faltantes.

Primero revisemos los valores faltantes

In [326]:
# Calcula la cantidad de nulos por columna
conteo_de_nulos_por_columna = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])

print(f"cantidad de filas: {df.count()}")
# Muestra el resultado
conteo_de_nulos_por_columna.show()

cantidad de filas: 66013
+--------+---------+----+-----+--------+----+-----------+-------+-------+---+--------+---------------+--------+-----+----+------------+-----+-----+-----+
|also_buy|also_view|asin|brand|category|date|description|details|feature|fit|imageURL|imageURLHighRes|main_cat|price|rank|similar_item|tech1|tech2|title|
+--------+---------+----+-----+--------+----+-----------+-------+-------+---+--------+---------------+--------+-----+----+------------+-----+-----+-----+
|       0|        0|   0|    0|       0|   0|          0|    534|      0|  0|       0|              0|       0|    0|   0|           0|    0|    0|    0|
+--------+---------+----+-----+--------+----+-----------+-------+-------+---+--------+---------------+--------+-----+----+------------+-----+-----+-----+



Podemos observar que la mayoria de las columnas no poseen faltantes, pero en un breve vistaso a nuestra tabla sabemos que esto no es cierto, por lo que convertiremos las filas vacias de todas las columnas de tipo array y todas las columnas de tipo string a nulos

In [327]:
columnas_array_a_verificar = ["also_buy", "also_view", "category", "description", "feature"]

# Verifica si las columnas string están vacías y las marca como nulas si es necesario
for columna in columnas_array_a_verificar:
    df = df.withColumn(columna, when(size(col(columna)) > 0, col(columna)).otherwise(None))

columnas_string_a_verificar = ["asin", "title", "price", "rank", "brand", 'date', 'main_cat', 'similar_item', 'fit']

# Verifica si las columnas string están vacías y las marca como nulas si es necesario
for columna in columnas_string_a_verificar:
    df = df.withColumn(columna, when((length(col(columna)) == 0) | (col(columna).isNull()), None).otherwise(col(columna)))


In [328]:
# Calcula la cantidad de nulos por columna
conteo_de_nulos_por_columna = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])

print(df.count())
# Muestra el resultado
conteo_de_nulos_por_columna.show()

66013
+--------+---------+----+-----+--------+-----+-----------+-------+-------+-----+--------+---------------+--------+-----+----+------------+-----+-----+-----+
|also_buy|also_view|asin|brand|category| date|description|details|feature|  fit|imageURL|imageURLHighRes|main_cat|price|rank|similar_item|tech1|tech2|title|
+--------+---------+----+-----+--------+-----+-----------+-------+-------+-----+--------+---------------+--------+-----+----+------------+-----+-----+-----+
|   57580|    53714|   0|13325|   66006|66008|      32979|    534|  65924|66013|       0|              0|       0|25902|   0|       66013|    0|    0|  513|
+--------+---------+----+-----+--------+-----+-----------+-------+-------+-----+--------+---------------+--------+-----+----+------------+-----+-----+-----+



In [329]:
# Calcula el total de filas en el DataFrame
total_de_filas = df.count()

# Calcula la cantidad de nulos por columna
conteo_de_nulos_por_columna = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])

# Calcula el porcentaje de nulos respecto al total de filas para cada columna
porcentaje_de_nulos_por_columna = conteo_de_nulos_por_columna.select([((col(c) / total_de_filas) * 100).alias(c) for c in df.columns])

# Muestra el resultado
porcentaje_de_nulos_por_columna.show()

+-----------------+-----------------+----+----------------+-----------------+-----------------+-----------------+------------------+-----------------+-----+--------+---------------+--------+------------------+----+------------+-----+-----+------------------+
|         also_buy|        also_view|asin|           brand|         category|             date|      description|           details|          feature|  fit|imageURL|imageURLHighRes|main_cat|             price|rank|similar_item|tech1|tech2|             title|
+-----------------+-----------------+----+----------------+-----------------+-----------------+-----------------+------------------+-----------------+-----+--------+---------------+--------+------------------+----+------------+-----+-----+------------------+
|87.22524351264144|81.36882129277566| 0.0|20.1854180237226|99.98939602805508|99.99242573432505|49.95834153878781|0.8089315740838926|99.86517807098602|100.0|     0.0|            0.0|     0.0|39.237725902473755| 0.0|       10

Viendo nuestros reales valores faltantes tomamos la decision de hacer algo con esto, esto haremos con las siguiente columnas:
- also_buy: conservamos la columna y convertiremos los faltantes en N/D ya que la poca informacion que tiene es relevante
- also_view: similar a also_buy
- asin: indica el id de cada metadata por lo que la conservamos
- brand: conservamos y transformamos faltantes en N/D ya que es relevante
- category: eliminamos la columna por un 99% de valores nulos
- date: eliminamos por casi un 100% de valores faltantes
- description: reemplazamos faltantes por N/D por su posible uso
- details: trataremos esta columna mas adelante ya que extraeremos un dataframe de esta, pero eliminamos los valores nulos
- feature: similar a category y date
- fit: se elimina por 100% de valores faltantes
- imageURL: no es relevante por lo que eliminamos
- imageURLHighRes: similar a imageURL
- main_cat: se conserva ya que indica la categoria.
- price: se conserva y se trataran los faltantes tras un posterior analisis
- rank: se conserva por su relevancia en el ranking 
- similar_item: similar a fit, se elimina
- tech1 y tech2: si bien no figuran faltantes, tampoco son relevantes.
- title: se eliminaran los valores nulos unicamente

In [339]:
# Lista de nombres de columnas que deseas eliminar
columnas_a_eliminar = ["imageURL", "imageUrlHighRes", "tech1", 'tech2', 'category', 'date', 'feature', 'fit', 'similar_item']

# Elimina las columnas especificadas
df = df.drop(*columnas_a_eliminar)


# eliminamos los valores nulos
df = df.na.drop(subset=['title'])


In [331]:
df.printSchema()

root
 |-- also_buy: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- also_view: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- asin: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- description: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- details: struct (nullable = true)
 |    |-- \n    Item Weight: \n    : string (nullable = true)
 |    |-- \n    Product Dimensions: \n    : string (nullable = true)
 |    |-- ASIN:: string (nullable = true)
 |    |-- ASIN: : string (nullable = true)
 |    |-- Apparel: string (nullable = true)
 |    |-- Audio CD: string (nullable = true)
 |    |-- Audio Cassette: string (nullable = true)
 |    |-- Blu-ray Audio: string (nullable = true)
 |    |-- DVD: string (nullable = true)
 |    |-- DVD Audio: string (nullable = true)
 |    |-- Label:: string (nullable = true)
 |    |-- MP3 Music: string (nullable = true)
 |    |-- Note on Boxed Sets:: string

In [333]:
df = df.drop(*['category', 'date', 'similar_item', 'feature', 'fit'])

df.show(5)

+------------+------------+----------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+
|    also_buy|   also_view|      asin|               brand|         description|             details|            main_cat|price|                rank|               title|
+------------+------------+----------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+
|        null|        null|1415833044|Bill &amp; Gloria...|[Track Listings:\...|                null|<img src="https:/...|$5.94|967,938 in CDs &a...|<span class="a-si...|
|[1570199760]|[1932806105]|1570199329|Original Radio Br...|[With a style bui...|                null|<img src="https:/...| null|525,467 in CDs &a...|Suspense Classics...|
|        null|        null|1611141028|                null|                null|                null|<img src="https:/...| null|2,786,515 in CDs 

In [334]:
df.printSchema()

root
 |-- also_buy: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- also_view: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- asin: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- description: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- details: struct (nullable = true)
 |    |-- \n    Item Weight: \n    : string (nullable = true)
 |    |-- \n    Product Dimensions: \n    : string (nullable = true)
 |    |-- ASIN:: string (nullable = true)
 |    |-- ASIN: : string (nullable = true)
 |    |-- Apparel: string (nullable = true)
 |    |-- Audio CD: string (nullable = true)
 |    |-- Audio Cassette: string (nullable = true)
 |    |-- Blu-ray Audio: string (nullable = true)
 |    |-- DVD: string (nullable = true)
 |    |-- DVD Audio: string (nullable = true)
 |    |-- Label:: string (nullable = true)
 |    |-- MP3 Music: string (nullable = true)
 |    |-- Note on Boxed Sets:: string

In [335]:
df = df.withColumn("price", regexp_replace(col("price"), "\\$", "").cast("float"))

# Paso 1: Calcular el promedio de los valores no nulos en la columna 'price'
promedio_price = df.select(avg(col("price"))).first()[0]

# Calcular el rango intercuartil (IQR) de la columna 'price'
q1 = df.approxQuantile("price", [0.25], 0.01)[0]
q3 = df.approxQuantile("price", [0.75], 0.01)[0]
iqr = q3 - q1

# Definir un umbral para identificar outliers (por ejemplo, 1.5 veces el IQR)
umbral_superior = q3 + 1.5 * iqr
umbral_inferior = q1 - 1.5 * iqr

# Identificar los outliers en la columna 'price'
outliers = df.filter((col("price") > umbral_superior) | (col("price") < umbral_inferior))

# Mostrar los outliers
outliers.select("price").show()

# Paso 2: Rellenar los valores nulos en la columna 'price' con el promedio
#df = df.fillna(promedio_price, subset=["price"])


+------+
| price|
+------+
| 59.87|
|143.98|
| 52.58|
| 100.0|
| 86.46|
|  65.0|
| 53.22|
| 85.36|
|135.12|
| 54.79|
| 99.95|
| 54.94|
|149.98|
| 58.95|
| 78.95|
|  95.0|
|  80.0|
| 70.09|
| 99.92|
|154.38|
+------+
only showing top 20 rows



In [336]:
df = df.withColumn('main_cat', lit('Digital Music'))

In [337]:
# Utiliza regexp_replace para eliminar todos los caracteres que no sean números
df = df.withColumn('rank', regexp_replace(df['rank'], '[^0-9]', ''))

# Convierte la columna 'rank' a tipo Integer
df = df.withColumn('rank', df['rank'].cast('integer'))

In [338]:
fila = df.collect()[0]
fila['title']

'<span class="a-size-medium a-color-secondary a-text-normal"'